In [1]:
! pip install classification-models-3D

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 218.6 kB/s eta 0:00:0000:0100:01


In [2]:
!pip install keras_applications

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 183.7 kB/s eta 0:00:00a 0:00:01


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import nibabel as nib
from tensorflow import keras
import tensorflow_addons as tfa
import random
from scipy import ndimage
from tensorflow import keras
import os

In [4]:
df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")

In [5]:
n_image_list = np.arange(0, df.shape[0])
random_n_image_list = np.random.choice(n_image_list, df.shape[0], replace=False)

In [6]:
def data_gen():
    df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
    for i in random_n_image_list[:1800]:
        path = "../input/cervical-ct-preprocessing/" + str(df.iloc[i]["StudyInstanceUID"]) +".nii.gz"
        try:
            ct = nib.load(path)
            ct = ct.get_fdata()
            for pic in range(15,45):
                yield ({"input_1": np.stack([ct[:,:,pic],ct[:,:,pic],ct[:,:,pic]],axis=-1)}, 
                       {'co': df.iloc[i]["patient_overall"], 
                                        'c1': df.iloc[i]["C1"],
                                        'c2': df.iloc[i]["C2"],
                                        'c3': df.iloc[i]["C3"],
                                        'c4': df.iloc[i]["C4"],
                                        'c5': df.iloc[i]["C5"],
                                        'c6': df.iloc[i]["C6"],
                                        'c7': df.iloc[i]["C7"],})
                
        except:
            continue

            
            
def test_gen():
    df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
    for i in random_n_image_list[1800:]:
        path = "../input/cervical-ct-preprocessing/" + str(df.iloc[i]["StudyInstanceUID"]) +".nii.gz"
        try:
            ct = nib.load(path)
            ct = ct.get_fdata()
            for pic in range(15,45):
                yield ({"input_1": np.stack([ct[:,:,pic],ct[:,:,pic],ct[:,:,pic]],axis=-1)}, 
                       {'co': df.iloc[i]["patient_overall"], 
                                        'c1': df.iloc[i]["C1"],
                                        'c2': df.iloc[i]["C2"],
                                        'c3': df.iloc[i]["C3"],
                                        'c4': df.iloc[i]["C4"],
                                        'c5': df.iloc[i]["C5"],
                                        'c6': df.iloc[i]["C6"],
                                        'c7': df.iloc[i]["C7"],})
        except:
            continue

In [7]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([100,100,3])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

testset = tf.data.Dataset.from_generator(
     test_gen,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([100,100,3])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

2022-10-07 20:39:04.776322: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 20:39:04.873760: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 20:39:04.874554: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 20:39:04.877308: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [8]:
dataset = (
    dataset.batch(40)
    .prefetch(1)
)

testset = (
    testset.batch(40)
    .prefetch(1)
)

In [9]:
res = tf.keras.applications.ResNet101(
    include_top=False,
    weights="imagenet",
    input_shape=(100,100,3),
)

171458560/171446536 [==============================] - 1s 0us/step


In [10]:
inputs = keras.Input((100, 100, 3), name = "input_1")

features = res(inputs)

x = keras.layers.Flatten()(features)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(8, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)

outputo = keras.layers.Dense(1 ,activation = 'sigmoid', name="co")(x)
output1 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c1")(x)
output2 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c2")(x)
output3 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c3")(x)
output4 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c4")(x)
output5 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c5")(x)
output6 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c6")(x)
output7 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c7")(x)
                                               
model = keras.models.Model(inputs=inputs, outputs=[outputo,output1,output2,output3,output4,output5,output6,output7, features])

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
resnet101 (Functional)          (None, 4, 4, 2048)   42658176    input_1[0][0]                    
__________________________________________________________________________________________________
flatten (Flatten)               (None, 32768)        0           resnet101[0][0]                  
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32768)        131072      flatten[0][0]                    
______________________________________________________________________________________________

In [32]:
for i in dataset.take(1):
    print(np.array(res(i[0]["input_1"][:1])).shape)

(1, 4, 4, 2048)


Exception ignored in: <generator object data_gen at 0x7f3dbef74d50>
RuntimeError: generator ignored GeneratorExit


In [22]:
opt = tfa.optimizers.AdamW(
    weight_decay=2e-4,
    learning_rate = 0.0009,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    label_smoothing=0.1, 
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)
metrics = [tf.keras.metrics.AUC()]
model.compile(optimizer=opt,
              loss={"co": loss, 
                   "c1": loss,
                   "c2": loss,
                   "c3": loss,
                   "c4": loss,
                   "c5": loss,
                   "c6": loss,
                   "c7": loss,},
                metrics={"co": metrics, 
                   "c1": metrics,
                   "c2": metrics,
                   "c3": metrics,
                   "c4": metrics,
                   "c5": metrics,
                   "c6": metrics,
                   "c7": metrics,})


In [23]:
!mkdir best_model

In [24]:
model_checkpoint_callback_LASSO = tf.keras.callbacks.ModelCheckpoint(
    filepath = './best_model/model',
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

In [25]:
history = model.fit(dataset, validation_data = testset, epochs=1, callbacks=[model_checkpoint_callback_LASSO])

1350/1350 [==============================] - 263s 182ms/step - loss: 4.0785 - co_loss: 0.6992 - c1_loss: 0.4426 - c2_loss: 0.5425 - c3_loss: 0.3777 - c4_loss: 0.4404 - c5_loss: 0.4448 - c6_loss: 0.5336 - c7_loss: 0.5978 - c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.6758 - val_loss: 3.4767 - val_co_loss: 0.6943 - val_c1_loss: 0.3391 - val_c2_loss: 0.4269 - val_c3_loss: 0.2716 - val_c4_loss: 0.3040 - val_c5_loss: 0.4186 - val_c6_loss: 0.4687 - val_c7_loss: 0.5534 - val_c7_c6_c5_c4_c3_c2_c1_co_auc_1: 0.7612


In [39]:
def data_gen2():
    df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
    for i in random_n_image_list[:1800]:
        path = "../input/cervical-ct-preprocessing/" + str(df.iloc[i]["StudyInstanceUID"]) +".nii.gz"
        try:
            ct = nib.load(path)
            ct = ct.get_fdata()
            
            yield ({"input_1": ct}, 
                   {'co': df.iloc[i]["patient_overall"], 
                                    'c1': df.iloc[i]["C1"],
                                    'c2': df.iloc[i]["C2"],
                                    'c3': df.iloc[i]["C3"],
                                    'c4': df.iloc[i]["C4"],
                                    'c5': df.iloc[i]["C5"],
                                    'c6': df.iloc[i]["C6"],
                                    'c7': df.iloc[i]["C7"],})
                
        except:
            continue

            
            
def test_gen2():
    df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
    for i in random_n_image_list[1800:]:
        path = "../input/cervical-ct-preprocessing/" + str(df.iloc[i]["StudyInstanceUID"]) +".nii.gz"
        try:
            ct = nib.load(path)
            ct = ct.get_fdata()
           
            yield ({"input_1": ct}, 
                   {'co': df.iloc[i]["patient_overall"], 
                                    'c1': df.iloc[i]["C1"],
                                    'c2': df.iloc[i]["C2"],
                                    'c3': df.iloc[i]["C3"],
                                    'c4': df.iloc[i]["C4"],
                                    'c5': df.iloc[i]["C5"],
                                    'c6': df.iloc[i]["C6"],
                                    'c7': df.iloc[i]["C7"],})
        except:
            continue

In [40]:
dataset2 = tf.data.Dataset.from_generator(
     data_gen2,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([100,100,60])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

testset2 = tf.data.Dataset.from_generator(
     test_gen2,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([100,100,60])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

In [41]:
dataset2 = (
    dataset2.batch(1)
    .prefetch(1)
)

testset2 = (
    testset2.batch(1)
    .prefetch(1)
)

In [159]:
inputs = keras.Input((100, 100, 60),name = "input_1")

extracted_features = res(tf.stack([inputs,inputs,inputs],axis=-1)[...,15,:])
for pic in range(16,45):
    img = tf.stack([inputs,inputs,inputs],axis=-1)[...,pic,:]
    pred = res(img)

    extracted_features = tf.concat([extracted_features, pred], axis=0)

s = keras.layers.Reshape((-1,2048))(extracted_features)
s = keras.layers.Dense(20, activation='relu')(s)
s = keras.layers.LSTM(10, return_sequences=True,go_backwards=True)(s)
s = keras.layers.Dense(10, activation='relu')(s)
s = keras.layers.BatchNormalization()(s)
s = keras.layers.LSTM(10, return_sequences=False,go_backwards=True)(s)
outputo = keras.layers.Dense(1 ,activation = 'sigmoid', name="co")(s)
output1 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c1")(s)
output2 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c2")(s)
output3 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c3")(s)
output4 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c4")(s)
output5 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c5")(s)
output6 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c6")(s)
output7 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c7")(s)
                                               
s_model = keras.models.Model(inputs=inputs, outputs=[outputo,output1,output2,output3,output4,output5,output6,output7])

In [160]:
s_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 60 0                                            
__________________________________________________________________________________________________
tf.stack_262 (TFOpLambda)       (None, 100, 100, 60, 0           input_1[0][0]                    
                                                                 input_1[0][0]                    
                                                                 input_1[0][0]                    
__________________________________________________________________________________________________
tf.stack_263 (TFOpLambda)       (None, 100, 100, 60, 0           input_1[0][0]                    
                                                                 input_1[0][0]              

In [161]:
opt = tfa.optimizers.AdamW(
    weight_decay=2e-4,
    learning_rate = 0.0009,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    label_smoothing=0.1, 
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)
metrics = [tf.keras.metrics.AUC()]
s_model.compile(optimizer=opt,
              loss={"co": loss, 
                   "c1": loss,
                   "c2": loss,
                   "c3": loss,
                   "c4": loss,
                   "c5": loss,
                   "c6": loss,
                   "c7": loss,},
                metrics={"co": metrics, 
                   "c1": metrics,
                   "c2": metrics,
                   "c3": metrics,
                   "c4": metrics,
                   "c5": metrics,
                   "c6": metrics,
                   "c7": metrics,})

In [162]:
!mkdir final_model

mkdir: cannot create directory ‘final_model’: File exists


In [163]:
model_checkpoint_callback_LASSO = tf.keras.callbacks.ModelCheckpoint(
    filepath = './final_model/model',
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

In [164]:
history = s_model.fit(dataset2, validation_data = testset2, epochs=1, callbacks=[model_checkpoint_callback_LASSO])

Exception ignored in: <generator object data_gen2 at 0x7f3da7c56f50>
RuntimeError: generator ignored GeneratorExit
2022-10-07 22:27:30.436060: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at segment_reduction_ops_impl.h:425 : Invalid argument: data.shape = [1] does not start with segment_ids.shape = [30]
2022-10-07 22:27:30.436845: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at segment_reduction_ops_impl.h:425 : Invalid argument: data.shape = [1] does not start with segment_ids.shape = [30]
2022-10-07 22:27:30.437314: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at segment_reduction_ops_impl.h:425 : Invalid argument: data.shape = [1] does not start with segment_ids.shape = [30]
2022-10-07 22:27:30.437564: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at segment_reduction_ops_impl.h:425 : Invalid argument: data.shape = [1] does not start with segment_ids.shape = [30]
2022-10-07 22:27:30.437695: W tensorf

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  data.shape = [1] does not start with segment_ids.shape = [30]
	 [[node UnsortedSegmentSum_1 (defined at tmp/ipykernel_17/2853456855.py:1) ]]
	 [[ConstantFolding/assert_greater_equal/Assert/AssertGuard/switch_pred/_2_const_false/_79]]
  (1) Invalid argument:  data.shape = [1] does not start with segment_ids.shape = [30]
	 [[node UnsortedSegmentSum_1 (defined at tmp/ipykernel_17/2853456855.py:1) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_1836456]

Function call stack:
train_function -> train_function
